# North American Einstein Toolkit School 2021: NRPy+ Tutorial
### Authors: Leo Werneck, Terrence Pierre-Jacques, Patrick Nelson, Zach Etienne, & Thiago Assumpção

<table><tr>
    <td style="background-color: white;">
        <a href=http://nrpyplus.net>
            <img src=https://github.com/leowerneck/NRPy_Tutorial_ETK_Workshop_2021/blob/main/logos/Nerpy.png?raw=true style="width: 150px;" />
        </a>
     </td>
    <td style="background-color: white;"> 
        <a href=https://www.wvu.edu>
            <img src=https://github.com/leowerneck/NRPy_Tutorial_ETK_Workshop_2021/blob/main/logos/WVU.png?raw=true style="width: 250px;"/>
        </a>
    </td>
    <td style="background-color: white;">
        <a href=https://www.uidaho.edu>
            <img src=https://github.com/leowerneck/NRPy_Tutorial_ETK_Workshop_2021/blob/main/logos/Idaho.png?raw=true style="width: 250px;"/>
        </a>
    </td>
    <td style="background-color: white;">
        <a href=https://einsteintoolkit.org>
            <img src=https://github.com/leowerneck/NRPy_Tutorial_ETK_Workshop_2021/blob/main/logos/ETK.png?raw=true style="width: 125px;"/>
        </a>
    </td>
    <td style="background-color: white;">
        <a href=https://www.nsf.gov>
            <img src=https://github.com/leowerneck/NRPy_Tutorial_ETK_Workshop_2021/blob/main/logos/NSF.jpg?raw=true style="width: 125px;"/>
        </a>
    </td>
</tr></table>

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This tutorial notebook is organized as follows:

0. [Step 0](#introduction): **Introduction**
    1. [Step 0.a](#maxwell_equations): Maxwell's equations in vacuum, flat space, and Cartesian coordinates
    1. [Step 0.b](#potential_formulation_maxwell_equations): Potential formulation of Maxwell's equations
    1. [Step 0.c](#hyperbolicity_maxwell_equations): Improving the hyperbolicity of Maxwell's equations
    1. [Step 0.d](#systems_I_and_II): Summary
1. [Step 1](#implementation): **Part I - Implementing Maxwell's equations using NRPy+**
    1. [Step 1.a](#initializenrpy): Initialize core Python/NRPy+ modules
    1. [Step 1.b](#gridfunction_declaration): Gridfunction declaration
    1. [Step 1.c](#finite_differences): Finite difference derivatives
    1. [Step 1.d](#system_I_implementation): Implementation of system I
        1. [Step 1.d.i](#system_I_evolution_eqs): *System I evolution equations*
        1. [Step 1.d.ii](#system_I_constraint_eqs): *System I constraint equation*
    1. [Step 1.e](#system_II_implementation): Implementation of system II
        1. [Step 1.e.i](#system_II_evolution_eqs): *System II evolution equations*
        1. [Step 1.e.ii](#system_II_constraint_eqs): *System II constraint equations*
1. [Step 2](#thorn_writing): **Part II - Einstein Toolkit thorn writing**
    1. [Step 2.a](#main_function): The thorn's driver/main function
    1. [Step 2.b](#ccl_files): Thorn configuration files
        1. [Step 2.b.i](#param_ccl): *Generating the `param.ccl` file*
        1. [Step 2.b.ii](#interface_ccl): *Generating the `interface.ccl` file*
        1. [Step 2.b.iii](#schedule_ccl): *Generating the `schedule.ccl` file*
        1. [Step 2.b.iv](#configuration_ccl): *Generating the `configuration.ccl` file*
        1. [Step 2.b.v](#make_code_defn): *Generating the `make.code.defn` file*
1. [Step 3](#results): **Part III - Results**
1. [Step 4](#latex_pdf_output): **Output this notebook to $\LaTeX$-formatted PDF file**

<a id='introduction'></a>

# Step 0: Introduction \[Back to [top](#toc)\]
$$\label{introduction}$$

In this tutorial we will provide a hands-on introduction on how to use the [NRPy+ infrastructure](http://nrpyplus.net) to write thorns for the [Einstein Toolkit (ETK)](https://einsteintoolkit.org). Although we will give an overview of NRPy+ and provide a high level description of some of its modules, we do not aim to thoroughly describe every aspect of the infrastructure. For more details on NRPy+'s inner workings, we refer the reader to:

* [NRPy+'s webpage](http://nrpyplus.net)
* [NRPy+'s extensive & pedagogical documentation](https://mybinder.org/v2/gh/zachetienne/nrpytutorial/master?filepath=NRPyPlus_Tutorial.ipynb)
* [NRPy+ tutorial from the 2020 ETK workshop by Zach Etienne](https://www.youtube.com/watch?v=TIPiW5-mPOM)

<hr style="width:100%;height:3px;color:black"/>

<a id='maxwell_equations'></a>

## Step 0.a: Maxwell's equations in vacuum, flat space, and Cartesian coordinates \[Back to [top](#toc)\]
$$\label{maxwell_equations}$$

In this tutorial we will generate an ETK thorn to solve [Maxwell's equations](https://en.wikipedia.org/wiki/Maxwell%27s_equations) in flat space using Cartesian coordinates. In [Gaussian](https://en.wikipedia.org/wiki/Gaussian_units) and $c = 1$ units, the system of equations we are interested in is

$$
\begin{align}
\vec{\nabla} \cdot \vec{E} &= 0, \\
\vec{\nabla} \cdot \vec{B} &= 0, \\
\frac{\partial \vec{E}}{\partial t} &= \vec{\nabla} \times \vec{B}, \\
\frac{\partial \vec{B}}{\partial t} &= -\vec{\nabla} \times \vec{E},
\end{align}
$$

where $\vec{E}$ is the electric field, $\vec{B}$ is the magnetic field, and $\vec{\nabla}\cdot\vec{V}$ and $\vec{\nabla}\times\vec{V}$ are the divergence and the curl of the vector $\vec{V}$, respectively.

The last two equations involve time derivatives of $\vec{E}$ and $\vec{B}$ and therefore are referred to as *evolution* equations. The first two equations must be satisfied for all times, and are referred to as *constraint* equations.

<a id='potential_formulation_maxwell_equations'></a>

## Step 0.b: Potential formulation of Maxwell's equations \[Back to [top](#toc)\]
$$\label{potential_formulation_maxwell_equations}$$

A formulation of Maxwell's equations that is particularly useful for numerical integration can be found by introducing auxiliary variables. We start by introducing a new vector quantity, $\vec{A}$, known as the *magnetic* or *vector* potential, such that

$$
\vec{B} = \vec{\nabla}\times\vec{A}.
$$

Note that the "no-magnetic monopole" constraint is automatically satisfied:

$$
\vec{\nabla} \cdot \vec{B} = \vec{\nabla} \cdot \bigl(\vec{\nabla}\times\vec{A}\bigr) = 0.
$$

[Ampère's law](https://en.wikipedia.org/wiki/Ampère%27s_circuital_law), which is the evolution equation for $\vec{B}$, can then be written as

$$
\frac{\partial}{\partial t}\bigl(\vec{\nabla}\times\vec{A}\bigr) = -\vec{\nabla} \times \vec{E} \implies \vec{\nabla}\times\left(\frac{\partial\vec{A}}{\partial t} + \vec{E}\right) = 0 \implies \frac{\partial\vec{A}}{\partial t} + \vec{E} = -\vec{\nabla}\Phi,
$$

where $\Phi$ is an arbitrary scalar function known as the *electric* (or *scalar*) potential. Finally, [Faraday's law](https://en.wikipedia.org/wiki/Faraday%27s_law_of_induction), the evolution equation for $\vec{E}$, can be written as

$$
\frac{\partial\vec{E}}{\partial t} = \vec{\nabla}\times\bigl(\vec{\nabla}\times\vec{A}\bigr) = -\nabla^{2}\vec{A} + \vec{\nabla}\bigl(\vec{\nabla}\cdot\vec{A}\bigr).
$$

Thus Maxwell's equations now read

$$
\begin{align}
\vec{\nabla} \cdot \vec{E} &= 0, \\
\frac{\partial\vec{E}}{\partial t} &= -\nabla^{2}\vec{A} + \vec{\nabla}\bigl(\vec{\nabla}\cdot\vec{A}\bigr),\\
\frac{\partial\vec{A}}{\partial t} &= -\vec{E} - \vec{\nabla}\Phi.
\end{align}
$$

Note that we now end up with 7 dynamical fields, namely $\left(\Phi,\vec{A},\vec{E}\right)$, but only 6 evolution equations. We can remedy this by adopting a particular gauge. We will choose here the [Lorenz gauge](https://en.wikipedia.org/wiki/Lorenz_gauge_condition), which reads

$$
\frac{\partial\Phi}{\partial t} = -\vec{\nabla}\cdot\vec{A}.
$$

Thus we arrive at the evolution equations

$$
\boxed{
\begin{align}
\frac{\partial\vec{E}}{\partial t} &= -\nabla^{2}\vec{A} + \vec{\nabla}\bigl(\vec{\nabla}\cdot\vec{A}\bigr)\\
\frac{\partial\vec{A}}{\partial t} &= -\vec{E} - \vec{\nabla}\Phi\\
\frac{\partial\Phi}{\partial t} &= -\vec{\nabla}\cdot\vec{A}
\end{align}
}\ ,
$$

plus the constraints

$$
\boxed{ \vec{\mathcal{C}} \equiv \vec{\nabla} \cdot \vec{E} = 0}\ ,
$$

which must be satisfied at all times.

<a id='hyperbolicity_maxwell_equations'></a>

## Step 0.c: Improving the hyperbolicity of Maxwell's equations \[Back to [top](#toc)\]
$$\label{hyperbolicity_maxwell_equations}$$

If we take a time derivative of the evolution equation for the vector potential and plug in the evolution equation for the electric field and the scalar potential we find

$$
\frac{\partial^{2}\vec{A}}{\partial t^{2}} = -\frac{\partial\vec{E}}{\partial t} - \vec{\nabla}\left(\frac{\partial\Phi}{\partial t}\right) = \nabla^{2}\vec{A} - \vec{\nabla}\bigl(\vec{\nabla}\cdot\vec{A}\bigr) - \vec{\nabla}\left(\frac{\partial\Phi}{\partial t}\right).
$$

Notice that this is almost a wave equation for the vector potential $\vec{A}$, but we have a mixed derivative term on the right-hand side that spoils this behaviour. We can thus introduce a new auxiliary variable defined as

$$
\Gamma \equiv \vec{\nabla}\cdot\vec{A},
$$

so that we have

$$
\frac{\partial^{2}\vec{A}}{\partial t^{2}} = \nabla^{2}\vec{A} - \vec{\nabla}\Gamma - \vec{\nabla}\left(\frac{\partial\Phi}{\partial t}\right).
$$

The resulting system of equations now becomes

$$
\boxed{
\begin{align}
\frac{\partial\vec{E}}{\partial t} &= -\nabla^{2}\vec{A} + \vec{\nabla}\Gamma\\
\frac{\partial\vec{A}}{\partial t} &= -\vec{E} - \vec{\nabla}\Phi\\
\frac{\partial\Phi}{\partial t} &= -\Gamma\\
\frac{\partial\Gamma}{\partial t} &= -\nabla^{2}\Phi
\end{align}
}\ ,
$$

while the constraints that must satisfied are

$$
\boxed{
\begin{align}
\mathcal{C} &\equiv \vec{\nabla} \cdot \vec{E} = 0\\
\mathcal{G} &\equiv \Gamma - \vec{\nabla}\cdot\vec{A} = 0
\end{align}
}\ .
$$

<a id='systems_I_and_II'></a>

## Step 0.d: Summary \[Back to [top](#toc)\]
$$\label{systems_I_and_II}$$

Switching to index notation, we write

$$
\frac{\partial}{\partial t} \equiv \partial_{t}\ ;\ \frac{\partial}{\partial x^{i}} \equiv \partial_{i},
$$

where $\vec{x} \equiv x^{i} = (x,y,z)$.

We are thus interested in solving Maxwell's equations in vacuum (i.e. without source terms), in flat space, and in Cartesian coordinates, and we will do so using:

$$
\text{System I:}\ 
\boxed{
\begin{align}
\color{blue}{\partial_{t}E^{i}} &\, \color{blue}{= -\partial^{j}\partial_{j}A^{i} + \partial^{i}\partial_{j}A^{j}}\\
\color{blue}{\partial_{t}A^{i}} &\, \color{blue}{= -E^{i} - \partial^{i}\Phi}\\
\color{blue}{\partial_{t}\Phi}  &\, \color{blue}{= -\partial_{i}A^{i}}\\
\color{red}{\partial_{i}E^{i}} &\, \color{red}{= 0}
\end{align}
}
\ ,
$$

and

$$
\text{System II:}\ 
\boxed{
\begin{align}
\color{blue}{\partial_{t}E^{i}}  &\, \color{blue}{= -\partial^{j}\partial_{j}A^{i} + \partial^{i}\Gamma}\\
\color{blue}{\partial_{t}A^{i}}  &\, \color{blue}{= -E^{i} - \partial^{i}\Phi}\\
\color{blue}{\partial_{t}\Phi}   &\, \color{blue}{= -\Gamma}\\
\color{blue}{\partial_{t}\Gamma} &\, \color{blue}{= -\partial^{i}\partial_{i}\Phi}\\
\color{red}{\partial_{i}E^{i}}   &\, \color{red}{= 0}\\
\color{red}{\Gamma}              &\, \color{red}{= \partial_{i}A^{i}}
\end{align}
}\ .
$$

In the colorcode above, $\color{blue}{\text{blue}}$ equations are the $\color{blue}{\text{evolution}}$ equations, while $\color{red}{\text{red}}$ equations are the $\color{red}{\text{constraint}}$ equations.

<hr style="width:100%;height:3px;color:black"/>

<a id='implementation'></a>

# Step 1: Part I - Implementing Maxwell's equations using NRPy+ \[Back to [top](#toc)\]
$$\label{implementation}$$

<a id='initializenrpy'></a>

## Step 1.a: Initialize core Python/NRPy+ modules \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

Let's start by importing all the needed modules from NRPy+:

In [1]:
# Step 1.a: Import core Python/NRPy+ modules
# Step 1.a.i: Import needed Python modules
import shutil, os, sys # Standard Python modules for multiplatform OS-level functions
import time            # Standard Python module; useful for benchmarking
import sympy as sp     # SymPy: a Python library for symbolic mathematics

# Step 1.a.ii: Add the "nrpy_core" directory to the path
base_dir      = os.getcwd()
nrpy_core_dir = os.path.join(base_dir,"nrpy_core")
sys.path.append(nrpy_core_dir)

# Step 1.b: Load needed NRPy+ modules
from outputC import lhrh        # NRPy+: Core C code output module
import finite_difference as fin # NRPy+: Finite difference C code generation module
import NRPy_param_funcs as par  # NRPy+: Parameter interface
import grid as gri              # NRPy+: Functions having to do with numerical grids
import loop as lp               # NRPy+: Generate C code loops
import indexedexp as ixp        # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import reference_metric as rfm  # NRPy+: Reference metric support
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface

<a id='gridfunction_declaration'></a>

## Step 1.b: Gridfunction declaration \[Back to [top](#toc)\]
$$\label{gridfunction_declaration}$$

When solving Maxwell's equations numerically using `NRPy`, we will *discretize* space as a *grid*. Assuming we choose a cube of length $L$ and discretize the $(x,y,z)$-directions using $(N_{x},N_{y},N_{z})$ points, respectively, we introduce the notation

$$
\begin{alignat}{2}
x \to x_{i} &\equiv x_{\rm min} + (i-0.5) \cdot \Delta x,\ &i=0,1,\ldots,N_{x}-1\\
y \to y_{j} &\equiv y_{\rm min} + (j-0.5) \cdot \Delta y,\ &j=0,1,\ldots,N_{y}-1\\
z \to z_{k} &\equiv z_{\rm min} + (k-0.5) \cdot \Delta z,\ &k=0,1,\ldots,N_{z}-1
\end{alignat}
$$

where $(\Delta x,\Delta y,\Delta z)$ are known as the *step sizes*. Our particular choice of discretization avoids the origin and is known as *cell-centered*.

A particular function $f(x,y,z)$ can be evaluated on the points in the grid. We introduce the notation

$$
f_{ijk} \equiv f(x_{i},y_{j},z_{k}) = f(i\cdot\Delta x,j\cdot\Delta y,k\cdot\Delta z).
$$

$f_{ijk}$, i.e. the function $f(x,y,z)$ evaluated at the points of our numerical grid, is referred to as a *gridfunction*. Notice that the indices $i,j,k$ in this case are *not* indices associated with spacetime, but simply provide a useful bookeeping of where we are in our numerical grid.

In order to implement systems I and II, we will need the following *gridfunctions*:

* The electric field, $E^{i} = \bigl(E^{x},E^{y},E^{z}\bigr)$;
* The vector potential, $A^{i} = \bigl(A^{x},A^{y},A^{z}\bigr)$;
* The scalar potential, $\Phi$;
* The auxiliary scalar, $\Gamma$, which is only used in system II.

This means we need a total of 7 gridfunctions for system I and 8 gridfunctions for system II. We will now learn how to register [`SymPy`](https://www.sympy.org) variables which correspond to the gridfunctions above.

Notice first that the gridfunctions above fall into two distinct categories: scalar gridfunctions, $(\Phi,\Gamma)$, and 3-vector gridfunctions, $(E^{i},A^{i})$. Because of this, their declaration within `NRPy+` is also slightly different.

><font size=4>Scalar gridfunctions are registered using the `NRPy+` module [`grid.py`](#FIXME)</font>

><font size=4>Indexed expression gridfunctions are registered using the `NRPy+` module [`indexedexp.py`](#FIXME)</font>

We handle contravariant ("Up") and covariant ("Down") indices by appending "U"'s and "D"'s to the names of indexed expression variables. For example, in standard `NRPy+` notation, the variable that represents the tensor $M^{ab}_{\ \ \ \ \ \ cd}$ should be defined using

$$
\underbrace{M^{ab}_{\ \ \ \ \ \ cd}}_{\text{Indexed expression}} \leftrightarrow \underbrace{\text{MUUDD}}_{\text{NRPy+ variable}}.
$$

Variables that represent 4-dimensional indexed expressions should have a "4" in their names. For example, the 4-dimensional indexed expression $V_{\mu}$ is declared as

$$
\underbrace{V_{\mu}}_{\text{Indexed expression}} \leftrightarrow \underbrace{\text{V4D}}_{\text{NRPy+ variable}}.
$$

In [13]:
# Step 1.b: Gridfunction declaration
def declare_MaxwellVacuum_gridfunctions_if_not_declared_already():
    # Step 1.b.i: This if statement simply prevents multiple
    #             declarations of the same gridfunctions,
    #             which would result in errors.
    for i in range(len(gri.glb_gridfcs_list)):
        if "Phi" in gri.glb_gridfcs_list[i].name:
            Phi,Gamma = sp.symbols("Phi Gamma",real=True)
            EU        = ixp.declarerank1("EU",DIM=3)
            AU        = ixp.declarerank1("AU",DIM=3)
            return Phi,Gamma,EU,AU

    # Step 1.b.i: Scalar gridfunctions: Phi and Gamma
    Phi,Gamma = gri.register_gridfunctions("EVOL",["Phi","Gamma"])

    # Step 1.b.ii: 3-vector gridfunctions: E^{i} and A^{i}
    EU = ixp.register_gridfunctions_for_single_rank1("EVOL","EU",DIM=3)
    AU = ixp.register_gridfunctions_for_single_rank1("EVOL","AU",DIM=3)

    return Phi,Gamma,EU,AU

Phi,Gamma,EU,AU = declare_MaxwellVacuum_gridfunctions_if_not_declared_already()

<a id='finite_differences'></a>

## Step 1.c: Finite difference derivatives \[Back to [top](#toc)\]
$$\label{finite_differences}$$

`NRPy+` handles derivatives numerically using [finite differences](https://en.wikipedia.org/wiki/Finite_difference). This happens when we are generating the C code from the symbolic expressions. Before we get to the actual C code generation, we need symbolic variables to represent these derivatives.

From the point of view of the programmer, a derivative of an object should be thought of as simply another indexed expression. For example, we can think of the second derivative of the vector $A^{i}$ as the new object $a^{k}_{\ \ ij}$, i.e.

$$
a^{k}_{\ \ ij} \equiv \partial_{i}\partial_{j}A^{k}.
$$

This means that the declaration of derivatives is also performed using the [`indexedexp.py`](FIXME) module. However, there is a special notation to distinguish "derivative indices" ($i$ and $j$ in the example above) from "regular indices" ($k$ in the example above). This distinction must be introduced so that the [`outputC.py`](FIXME) module correctly identifies derivatives and replaces them with the appropriate finite difference expression. This special notation consists of appending a "_d" (lowercase!) *before* the derivative indices. For example:

$$
\partial_{i}\partial_{j}A^{k} \equiv \underbrace{A^{k}_{\ ,ij}}_{\text{Indexed expression}} \leftrightarrow \underbrace{\text{AU_dDD}}_{\text{NRPy+ variable}}.
$$

The underscore preceding the derivative indices is introduced to distinguish *numerical* derivatives (i.e. those which we evaluate using finite differences) from *analytic* derivatives (i.e. those which we evaluate using `SymPy` and exact, symbolic differentiation).

Here we provide a simple example that evaluates the derivative of a scalar $\psi$ using finite differences. This variable will not be used again, so we prepend its name with an underscore to identify it as a dummy variable.

[Using fourth-order accurate finite differences, the expressions we should obtain for the first and second derivatives](https://en.wikipedia.org/wiki/Finite_difference_coefficient) of $\psi$ along the $x$-direction are, respectively,

$$
\begin{align}
\partial_{x}\psi &= \frac{1}{\Delta x}\left(\frac{1}{12}\psi_{i-2} - \frac{2}{3}\psi_{i-1} + \frac{2}{3}\psi_{i+1} - \frac{1}{12}\psi_{i+2}\right),\\
\partial_{x}^{2}\psi &= \frac{1}{\Delta x^{2}}\left(-\frac{1}{12}\psi_{i-2} + \frac{4}{3}\psi_{i-1} - \frac{5}{2}\psi_{i} + \frac{4}{3}\psi_{i+1} - \frac{1}{12}\psi_{i+2}\right).
\end{align}
$$

Let us now demonstrate that `NRPy+` gives us the correct expressions.

In [9]:
# Step 1.c: Finite differences
# Step 1.c.i: Declare the gridfunction psi and
#              its first and second derivatives
_psi     = gri.register_gridfunctions("EVOL","psi")
_psi_dD  = ixp.declarerank1("psi_dD")
_psi_dDD = ixp.declarerank2("psi_dDD","sym01") # <- symmetric under i <-> j

# Step 1.c.ii: Now get the finite difference expression
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER",4)

# Step 1.c.iii: Generate the C code
print(fin.FD_outputC("returnstring",
                     [lhrh(lhs="First__derivative",rhs=_psi_dD[0]),
                      lhrh(lhs="Second_derivative",rhs=_psi_dDD[0][0])],
                     params="outCverbose=False"))

# Step 1.c.iv: Remove the gridfunction psi from the list of gridfunctions
for gf in gri.glb_gridfcs_list:
    if gf.name == "psi":
        gri.glb_gridfcs_list.remove(gf)

{
   /*
    * NRPy+ Finite Difference Code Generation, Step 1 of 2: Read from main memory and compute finite difference stencils:
    */
   const double psi_i0m2_i1_i2 = in_gfs[IDX4(PSIGF, i0-2,i1,i2)];
   const double psi_i0m1_i1_i2 = in_gfs[IDX4(PSIGF, i0-1,i1,i2)];
   const double psi = in_gfs[IDX4(PSIGF, i0,i1,i2)];
   const double psi_i0p1_i1_i2 = in_gfs[IDX4(PSIGF, i0+1,i1,i2)];
   const double psi_i0p2_i1_i2 = in_gfs[IDX4(PSIGF, i0+2,i1,i2)];
   const double FDPart1_Rational_2_3 = 2.0/3.0;
   const double FDPart1_Rational_1_12 = 1.0/12.0;
   const double FDPart1_Rational_5_2 = 5.0/2.0;
   const double FDPart1_Rational_4_3 = 4.0/3.0;
   const double psi_dD0 = invdx0*(FDPart1_Rational_1_12*(psi_i0m2_i1_i2 - psi_i0p2_i1_i2) + FDPart1_Rational_2_3*(-psi_i0m1_i1_i2 + psi_i0p1_i1_i2));
   const double psi_dDD00 = ((invdx0)*(invdx0))*(FDPart1_Rational_1_12*(-psi_i0m2_i1_i2 - psi_i0p2_i1_i2) + FDPart1_Rational_4_3*(psi_i0m1_i1_i2 + psi_i0p1_i1_i2) - FDPart1_Rational_5_2*psi);
   /*
    

<a id='system_I_implementation'></a>

## Step 1.d: Implementation of system I \[Back to [top](#toc)\]
$$\label{system_I_implementation}$$

We now implement system I, derived in [Step 0.b](#potential_formulation_maxwell_equations) above, which read

$$
\text{System I:}\ 
\boxed{
\begin{align}
\color{blue}{\partial_{t}E^{i}} &\, \color{blue}{= -\partial^{j}\partial_{j}A^{i} + \partial^{i}\partial_{j}A^{j}}\\
\color{blue}{\partial_{t}A^{i}} &\, \color{blue}{= -E^{i} - \partial^{i}\Phi}\\
\color{blue}{\partial_{t}\Phi}  &\, \color{blue}{= -\partial_{i}A^{i}}\\
\color{red}{\partial_{i}E^{i}} &\, \color{red}{= 0}
\end{align}
}
\ .
$$

In the colorcode above, $\color{blue}{\text{blue}}$ equations are $\color{blue}{\text{evolution}}$ equations, while $\color{red}{\text{red}}$ equations are $\color{red}{\text{constraint}}$ equations.

<a id='system_I_evolution_eqs'></a>

### Step 1.d.i: System I evolution equations \[Back to [top](#toc)\]
$$\label{system_I_evolution_eqs}$$

We will now implement the evolution equations of system I, namely

$$
\begin{align}
\partial_{t}E^{i} &= -\partial^{j}\partial_{j}A^{i} + \partial^{i}\partial_{j}A^{j},\\
\partial_{t}A^{i} &= -E^{i} - \partial^{i}\Phi,\\
\partial_{t}\Phi  &= -\partial_{i}A^{i}.
\end{align}
$$

In [15]:
# Step 1.d: Implementation of system I
# Step 1.d.i: System I evolution equations
def MaxwellVacuum_system_I_evolution_equations():
    # Step 1.d.i.1: Declare Maxwell gridfunctions
    _Phi,_Gamma,_EU,_AU = declare_MaxwellVacuum_gridfunctions_if_not_declared_already()
    
    # Step 1.d.i.2: Declare all derivatives appearing on the
    #               right-hand sides of the evolution equations
    #               of system I
    AU_dDD = ixp.declarerank3("AU_dDD","sym12")
    AU_dD  = ixp.declarerank2("AU_dD","nosym")
    Phi_dD = ixp.declarerank1("Phi_dD")
    
    # Step 1.d.i.3: Right-hand side of E^{i}:
    #
    # partial_{t}E^{i} = -partial^{j}partial_{j}A^{i} + partial^{i}partial_{j}A^{j}
    ErhsU = ixp.zerorank1()
    for i in range(DIM):
        for j in range(DIM):
            ErhsU[i] += -AU_dDD[i][j][j] + AU_dDD[j][j][i]
            
    # Step 1.d.i.4: Right-hand side of A^{i}:
    #
    # partial_{t}A^{i} = -E^{i} - partial^{i}Phi
    ArhsU = ixp.zerorank1()
    for i in range(DIM):
        ArhsU[i] = -EU[i] - Phi_dD[i]
        
    # Step 1.d.i.5: Right-hand side of Phi:
    #
    # partial_{t}Phi = -partial_{i}A^{i}
    Phi_rhs = sp.sympify(0)
    for i in range(DIM):
        Phi_rhs += -AU_dD[i][i]
        
    return ErhsU,ArhsU,Phi_rhs

<a id='system_I_constraint_eqs'></a>

### Step 1.d.ii: System I constraint equation \[Back to [top](#toc)\]
$$\label{system_I_constraint_eqs}$$

We now implement the constraint equation

$$
\mathcal{C} = \partial_{i}E^{i}.
$$

Notice that $\mathcal{C}=0$ *analytically*, but this constraint *is not* enforced during an evolution. It can instead be used as a diagnostic of how well our numerical solution satisfies Maxwell's equations.

In [29]:
# Step 1.d.ii: System I constraint equation
def MaxwellVacuum_system_I_constraint_equation():
    # Step 1.d.ii.1: Declare Maxwell gridfunctions
    _Phi,_Gamma,_EU,_AU = declare_MaxwellVacuum_gridfunctions_if_not_declared_already()

    # Step 1.d.ii.2: Declare all derivatives appearing on the
    #                right-hand sides of the constraint equations
    #                of system I
    EU_dD = ixp.declarerank2("EU_dD", "nosym")

    # Step 1.d.ii.3: Constraint equation (Gauss' law in vacuum):
    #
    # C = partial_{i}E^{i}
    C = sp.sympify(0)
    for i in range(DIM):
        C += EU_dD[i][i]
        
    return C

<a id='system_II_implementation'></a>

## Step 1.e: Implementation of system II \[Back to [top](#toc)\]
$$\label{system_II_implementation}$$

We now implement system II, derived in [Step 0.c](#hyperbolicity_maxwell_equations) above, which read

$$
\text{System II:}\ 
\boxed{
\begin{align}
\color{blue}{\partial_{t}E^{i}}  &\, \color{blue}{= -\partial^{j}\partial_{j}A^{i} + \partial^{i}\Gamma}\\
\color{blue}{\partial_{t}A^{i}}  &\, \color{blue}{= -E^{i} - \partial^{i}\Phi}\\
\color{blue}{\partial_{t}\Phi}   &\, \color{blue}{= -\Gamma}\\
\color{blue}{\partial_{t}\Gamma} &\, \color{blue}{= -\partial^{i}\partial_{i}\Phi}\\
\color{red}{\partial_{i}E^{i}}   &\, \color{red}{= 0}\\
\color{red}{\Gamma}              &\, \color{red}{= \partial_{i}A^{i}}
\end{align}
}\ .
$$

In the colorcode above, $\color{blue}{\text{blue}}$ equations are $\color{blue}{\text{evolution}}$ equations, while $\color{red}{\text{red}}$ equations are $\color{red}{\text{constraint}}$ equations.

<a id='system_II_evolution_eqs'></a>

### Step 1.e.i: System II evolution equations \[Back to [top](#toc)\]
$$\label{system_II_evolution_eqs}$$

We will now implement the evolution equations of system II, namely

$$
\begin{align}
\partial_{t}E^{i}  &= -\partial^{j}\partial_{j}A^{i} + \partial^{i}\Gamma,\\
\partial_{t}A^{i}  &= -E^{i} - \partial^{i}\Phi,\\
\partial_{t}\Phi   &= -\Gamma,\\
\partial_{t}\Gamma &= -\partial^{i}\partial_{i}\Phi.
\end{align}
$$

In [22]:
# Step 1.e: Implementation of system II
# Step 1.e.i: System II evolution equations
def MaxwellVacuum_system_II_evolution_equations():
    # Step 1.e.i.1: Declare Maxwell gridfunctions
    _Phi,Gamma,_EU,_AU = declare_MaxwellVacuum_gridfunctions_if_not_declared_already()

    # Step 1.e.i.2: Declare all derivatives appearing on the
    #               right-hand sides of the evolution equations
    #               of system II
    AU_dDD   = ixp.declarerank3("AU_dDD","sym12")
    Phi_dD   = ixp.declarerank1("Phi_dD")
    Phi_dDD  = ixp.declarerank2("Phi_dDD","sym01")
    Gamma_dD = ixp.declarerank1("Gamma_dD")

    # Step 1.e.i.3: Right-hand side of E^{i}:
    #
    # partial_{t}E^{i} = -partial^{j}partial_{j}A^{i} + partial^{i}Gamma
    ErhsU = ixp.zerorank1()
    for i in range(DIM):
        ErhsU[i] += Gamma_dD[i]
        for j in range(DIM):
            ErhsU[i] -= AU_dDD[i][j][j]

    # Step 1.e.i.4: Right-hand side of A^{i}:
    #
    # partial_{t}A^{i} = -E^{i} - partial^{i}Phi
    ArhsU = ixp.zerorank1()
    for i in range(DIM):
        ArhsU[i] = -EU[i] - Phi_dD[i]

    # Step 1.e.i.5: Right-hand side of Phi:
    #
    # partial_{t}Phi = -Gamma
    Phi_rhs = -Gamma
    
    # Step 1.e.i.6: Right-hand side of Gamma:
    #
    # partial_{t}Gamma = -partial^{i}partial_{i}Phi
    Gamma_rhs = sp.sympify(0)
    for i in range(DIM):
        Gamma_rhs -= Phi_dDD[i][i]

    return ErhsU,ArhsU,Phi_rhs,Gamma_rhs

<a id='system_II_constraint_eqs'></a>

### Step 1.e.ii: System II constraint equations \[Back to [top](#toc)\]
$$\label{system_II_constraint_eqs}$$

We now implement the constraint equations

$$
\begin{align}
\mathcal{C} &= \partial_{i}E^{i},\\
\mathcal{G} &= \Gamma - \partial_{i}A^{i}.
\end{align}
$$

Notice that $\mathcal{C}=0$ and $\mathcal{G}=0$ *analytically*, but these constraints *are not* enforced during an evolution. They can instead be used as diagnostics of how well our numerical solution satisfies Maxwell's equations.

In [25]:
# Step 1.e.ii: System I constraint equations
def MaxwellVacuum_system_II_constraint_equations():
    # Step 1.e.ii.1: Declare Maxwell gridfunctions
    _Phi,_Gamma,_EU,_AU = declare_MaxwellVacuum_gridfunctions_if_not_declared_already()

    # Step 1.e.ii.2: Declare all derivatives appearing on the
    #                right-hand sides of the constraint equations
    #                of system II
    EU_dD = ixp.declarerank2("EU_dD", "nosym")
    AU_dD = ixp.declarerank2("AU_dD", "nosym")

    # Step 1.e.ii.3: Gauss' law in vacuum:
    #
    # C = partial_{i}E^{i}
    C = sp.sympify(0)
    for i in range(DIM):
        C += EU_dD[i][i]
        
    # Step 1.e.ii.4: Gamma-constraint:
    #
    # G = Gamma - partial_{i}A^{i}
    G = Gamma
    for i in range(DIM):
        G += -AU_dD[i][i]
        
    return C,G

<hr style="width:100%;height:3px;color:black"/>

<a id='thorn_writing'></a>

# Step 2: Part II - Einstein Toolkit thorn writing \[Back to [top](#toc)\]
$$\label{thorn_writing}$$

<a id='main_function'></a>

## Step 2.a: The thorn's driver/main function \[Back to [top](#toc)\]
$$\label{main_function}$$

<a id='ccl_files'></a>

## Step 2.b: Thorn configuration files \[Back to [top](#toc)\]
$$\label{ccl_files}$$

<a id='param_ccl'></a>

### Step 2.b.i: Generating the `param.ccl` file \[Back to [top](#toc)\]
$$\label{param_ccl}$$

<a id='interface_ccl'></a>

### Step 2.b.ii: Generating the `interface.ccl` file \[Back to [top](#toc)\]
$$\label{interface_ccl}$$

<a id='schedule_ccl'></a>

### Step 2.b.iii: Generating the `schedule.ccl` file \[Back to [top](#toc)\]
$$\label{schedule_ccl}$$

<a id='configuration_ccl'></a>

### Step 2.b.iv: Generating the `configuration.ccl` file \[Back to [top](#toc)\]
$$\label{configuration_ccl}$$

<a id='make_code_defn'></a>

### Step 2.b.v: Generating the `make.code.defn` file \[Back to [top](#toc)\]
$$\label{make_code_defn}$$

<hr style="width:100%;height:3px;color:black"/>

<a id='results'></a>

# Step 3: Part III - Results \[Back to [top](#toc)\]
$$\label{results}$$

<hr style="width:100%;height:3px;color:black"/>

<a id='latex_pdf_output'></a>

# Step 4: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[ETK_Workshop_2021-NRPy_tutorial.pdf](ETK_Workshop_2021-NRPy_tutorial.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [2]:
# Step 4: Generate a PDF version of this tutorial notebook
# Step 4.a: First copy the latex_nrpy_style.tplx from the
#           nrpy_core directory to the base directory
src_file = os.path.join(nrpy_core_dir,"latex_nrpy_style.tplx")
dst_file = os.path.join(base_dir     ,"latex_nrpy_style.tplx")
shutil.copyfile(src_file,dst_file)

# Step 4.b: Now generate the PDF file
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("ETK_Workshop_2021-NRPy_tutorial")

# Step 4.c: Clean up by removing the latex_nrpy_style.tplx from
#           the base directory
cmd.delete_existing_files(dst_file)

Created ETK_Workshop_2021-NRPy_tutorial.tex, and compiled LaTeX file to PDF
    file ETK_Workshop_2021-NRPy_tutorial.pdf
